In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.adaptive.enabled", True)

In [0]:
# Bucketing

In [0]:
from pyspark.sql import functions as F

In [0]:
df = (
    spark.range(start=1, end=10_000, numPartitions=5)
    .select("id", F.rand(10).alias("value"))
)
display(df)

In [0]:
%fs
ls 'dbfs:/mnt/rainbowstorage/'

In [0]:
dbutils.fs.rm("dbfs:/mnt/rainbowstorage/non_bucket_tables/non_bucket_table1", True)
dbutils.fs.rm("dbfs:/mnt/rainbowstorage/non_bucket_tables/non_bucket_table2", True)
dbutils.fs.rm("dbfs:/mnt/rainbowstorage/bucket_tables/", True)

In [0]:
df.write.format("parquet").mode("overwrite").saveAsTable("non_bucket_table1", path="dbfs:/mnt/rainbowstorage/non_bucket_tables/non_bucket_table1/")
df.write.format("parquet").mode("overwrite").saveAsTable("non_bucket_table2", path="dbfs:/mnt/rainbowstorage/non_bucket_tables/non_bucket_table2/")

In [0]:
df.write.format("parquet").mode("overwrite").bucketBy(numBuckets=5, col="id").sortBy("id").saveAsTable("bucket_table1", path="dbfs:/mnt/rainbowstorage/bucket_tables/bucket_table1/")

df.write.format("parquet").mode("overwrite").bucketBy(numBuckets=5, col="id").sortBy("id").saveAsTable("bucket_table2", path="dbfs:/mnt/rainbowstorage/bucket_tables/bucket_table2/")

In [0]:
non_buck_df1 = spark.read.format("parquet").load("dbfs:/mnt/rainbowstorage/non_bucket_tables/non_bucket_table1")
display(non_buck_df1)

In [0]:
non_buck_df2 = spark.read.format("parquet").load("dbfs:/mnt/rainbowstorage/non_bucket_tables/non_bucket_table2")
display(non_buck_df1)

In [0]:
non_buck_result_df = (
    non_buck_df1
    .join(
        non_buck_df2,
        on="id",
        how="left"
    )
)
non_buck_result_df.show()

In [0]:
buck_df1 = spark.read.format("parquet").load("dbfs:/mnt/rainbowstorage/bucket_tables/bucket_table1")
display(buck_df1)

In [0]:
buck_df2 = spark.read.format("parquet").load("dbfs:/mnt/rainbowstorage/bucket_tables/bucket_table2")
display(buck_df2)

In [0]:
buck_result_df = (
    buck_df1
    .join(
        buck_df2,
        on="id",
        how="inner"
    )
)
buck_result_df.show()

In [0]:
buck_result_df.explain()

In [0]:
result_df = (
    buck_df1
    .join(
        buck_df2,
        on="id",
        how="left"
    )
)
result_df.show()